In [1]:
#imports
import pandas as pd
import requests

In [88]:
#first 10 blocks
url = 'https://blockstream.info/api'
max_block_height = requests.get(url + '/blocks/tip/height').json()
ten_bloacks_url = 'https://blockstream.info/api/blocks/{}'.format(max_block_height)
ten_blocks_data = requests.get(ten_bloacks_url).json()
block_headers = pd.DataFrame.from_dict(ten_blocks_data)

In [89]:

#appending more data to block_headers
max_block_height -= 10
#currently 100 rows are added. can be extended.
for i in range(10):
    url = 'https://blockstream.info/api/blocks/{}'.format(max_block_height)
    ten_blocks_data = requests.get(ten_bloacks_url).json()
    temp_df = pd.DataFrame.from_dict(ten_blocks_data)
    block_headers = pd.concat([block_headers, temp_df], ignore_index=True)
    max_block_height -= 10

In [90]:
block_headers.head()

,id,height,version,timestamp,tx_count,size,weight,merkle_root,previousblockhash,mediantime,nonce,bits,difficulty
0,00000000000000000006ac6370e86cb22836d2fce874c7...,745453,736083972,1658121914,2557,1511963,3998294,5bc37b0f48c7202f15cc1543970241ee5cb4b3913539c2...,00000000000000000004fbac7cc9b9547e6d0517d8cd24...,1658119514,1980036673,386508719,29152798808271
1,00000000000000000004fbac7cc9b9547e6d0517d8cd24...,745452,620797952,1658120442,704,766862,1812326,2eb97ade1fb791cb420fd9fd578245d2c008b0aee91208...,000000000000000000028aaa6ca38aea90600de75fab0c...,1658118964,2314781819,386508719,29152798808271
2,000000000000000000028aaa6ca38aea90600de75fab0c...,745451,939515908,1658120077,318,142894,386212,04984dbae749b230742a937f5ba562770f6f8537300c02...,00000000000000000006ad94b4345203925320373d1573...,1658118339,2857041610,386508719,29152798808271
3,00000000000000000006ad94b4345203925320373d1573...,745450,710451200,1658119890,163,81130,208540,57b419965ba14e46f7863d0b067ea6a54a4adba1211d22...,0000000000000000000066ed599cdd2e5a3156c7088d03...,1658117543,1000743989,386508719,29152798808271
4,0000000000000000000066ed599cdd2e5a3156c7088d03...,745449,568557572,1658119811,507,470389,1346776,86a664105127fe95c3b1652fc39922e9b88a93ef3bc2bc...,00000000000000000006d5dad5b9de4d590f2c49e9e8ae...,1658116754,27892038,386508719,29152798808271


In [91]:
#creating coinbase_txs
coinbase_txs = pd.DataFrame(columns=['txid', 'block_hash', 'version', 'locktime', 'outputs'])
#list used to get addresses
address_list = []
for hash in list(block_headers.id):
    txid = requests.get('https://blockstream.info/api/block/{}/txids'.format(hash)).json()[0]
    coinbase_data = requests.get('https://blockstream.info/api/tx/{}'.format(txid)).json()
    new_row = [coinbase_data['txid'], coinbase_data['status']['block_hash'], coinbase_data['version'], coinbase_data['locktime'], coinbase_data['vout']]
    coinbase_txs.loc[len(coinbase_txs.index)] = new_row
    address_list.append(coinbase_data['vout'][0]['scriptpubkey_address'])

#creating known_addresses
#for address in address_list:
#    address_data = requests.get('https://blockstream.info/api/address/{}'.format(address))

In [92]:
coinbase_txs.head()

,txid,block_hash,version,locktime,outputs
0,41a68d7251ba34e0655b2699062671bf51aee6142d30c3...,00000000000000000006ac6370e86cb22836d2fce874c7...,1,1060755413,[{'scriptpubkey': '76a914c825a1ecf2a6830c44016...
1,b203ff462d992443c466f21d467061750e9a743a9cb73e...,00000000000000000004fbac7cc9b9547e6d0517d8cd24...,1,0,[{'scriptpubkey': '76a914e38dc292da91a9392fecc...
2,5de7fb8a9c0b0af4c14b9d6ad93311bea34ce34b03de00...,000000000000000000028aaa6ca38aea90600de75fab0c...,1,1518445152,[{'scriptpubkey': '76a914a8a3f4fbd6a9a7b8c6687...
3,a460fe90b59ee9bf20dd64c5d85a3fb4c8d446ba008d3a...,00000000000000000006ad94b4345203925320373d1573...,2,0,[{'scriptpubkey': '76a9145e9b23809261178723055...
4,dd4c444f4a93972032ee06a7e14b942573a519fe78de3b...,0000000000000000000066ed599cdd2e5a3156c7088d03...,1,1226182865,[{'scriptpubkey': '76a914c825a1ecf2a6830c44016...


In [93]:
#get /address/:address doesn't return desired features
requests.get('https://blockstream.info/api/address/{}'.format(address_list[0])).json()

{'address': '1KFHE7w8BhaENAswwryaoccDb6qcT6DbYY',
 'chain_stats': {'funded_txo_count': 80713,
  'funded_txo_sum': 533159351943227,
  'spent_txo_count': 80695,
  'spent_txo_sum': 533142350884709,
  'tx_count': 80478},
 'mempool_stats': {'funded_txo_count': 0,
  'funded_txo_sum': 0,
  'spent_txo_count': 0,
  'spent_txo_sum': 0,
  'tx_count': 0}}